In [1]:
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/data/python-00000-of-00001.jsonl
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/data/pythonanalytics-00000-of-00001.jsonl

In [2]:
# !wget https://huggingface.co/datasets/m-a-p/Code-Feedback/resolve/main/Code-Feedback.jsonl
# !wget https://huggingface.co/datasets/teknium/OpenHermes-2.5/resolve/main/openhermes2_5.json

In [3]:
import json

with open('prepared-openhermes.json') as fopen:
    filtered = json.load(fopen)
    
filtered = [r['conversations'][0]['value'] for r in filtered]
filtered = [r for r in filtered if len(r) > 10]

In [4]:
len(filtered)

82022

In [5]:
!mkdir openhermes

mkdir: cannot create directory ‘openhermes’: File exists


In [6]:
import docker

client = docker.from_env()
for c in client.containers.list(filters = {'name': f'conv-openhermes-*'}):
    try:
        c.remove(force = True)
    except:
        pass
client.close()

In [7]:
from agent import ClientJupyterKernel, Agent, Generator
import os
import docker

def answer(q, i):
    filename = f'openhermes/{i}.json'
    if os.path.exists(filename):
        return
    
    jupyter_kernel_url = 'http://localhost:8081/execute'
    openai_base_url = 'http://localhost:8005/v1'
    model_name = 'xingyaoww/CodeActAgent-Mistral-7b-v0.1'

    generator = Generator(model_name, openai_base_url)
    prompt = q
    
    CONV_ID = None
    
    r = None
    agent = None
    for k in range(2):
        try:
            CONV_ID = f'openhermes-{i}-{k}'
            code_executor = ClientJupyterKernel(jupyter_kernel_url, CONV_ID)
            agent = Agent(generator, code_executor, conv_id=CONV_ID, silent = True)
            r = agent.run_prompt(prompt, n_max_executions = 10)
            agent.code_executor.shutdown()
            if '<execute>' in r[2]['content']:
                break
        except Exception as e:
            print(e)
            if agent is not None:
                agent.code_executor.shutdown()
            pass
        
    with open(filename, 'w') as fopen:
        json.dump({'original': q, 'result': r}, fopen)
    
    client = docker.from_env()
    for c in client.containers.list(filters = {'name': f'conv-openhermes-{i}-*'}):
        try:
            c.remove(force = True)
        except:
            pass
    client.close()

In [8]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [9]:
urls = [(q, no) for no, q in enumerate(filtered)]

In [10]:
# !rm code-feedback/210.json
answer(*urls[10003])

In [11]:
# !rm code-feedback/*.json

In [12]:
# with open('code-feedback/78.json') as fopen:
#     for r in json.load(fopen)['result']:
#         print(r)

In [13]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

  0%|                                                                                                                                        | 0/82022 [00:00<?, ?it/s]

New kernel created for conversation openhermes-1417-0
New kernel created for conversation openhermes-1413-0
New kernel created for conversation openhermes-1396-0
New kernel created for conversation openhermes-1402-0
New kernel created for conversation openhermes-1419-0
New kernel created for conversation openhermes-1394-0
New kernel created for conversation openhermes-1375-0
New kernel created for conversation openhermes-1381-0
New kernel created for conversation openhermes-1386-0
New kernel created for conversation openhermes-1395-0
New kernel created for conversation openhermes-1377-0
New kernel created for conversation openhermes-1373-0
New kernel created for conversation openhermes-1399-0
New kernel created for conversation openhermes-1411-0
New kernel created for conversation openhermes-1430-0
New kernel created for conversation openhermes-1410-0
New kernel created for conversation openhermes-1400-0
New kernel created for conversation openhermes-1387-0
New kernel created for conve

  2%|██▏                                                                                                                        | 1435/82022 [01:30<1:26:38, 15.50it/s]

New kernel created for conversation openhermes-1379-0


  2%|██▏                                                                                                                       | 1445/82022 [02:27<21:48:18,  1.03it/s]

...

 18%|█████████████████████▍                                                                                               | 15029/82022 [18:00:21<175:37:39,  9.44s/it]

New kernel created for conversation openhermes-15015-0
New kernel created for conversation openhermes-15005-1
New kernel created for conversation openhermes-14999-1
New kernel created for conversation openhermes-15022-0
New kernel created for conversation openhermes-15000-1
New kernel created for conversation openhermes-15023-0


 18%|█████████████████████▋                                                                                                | 15034/82022 [18:00:49<75:28:10,  4.06s/it]

New kernel created for conversation openhermes-15028-0
New kernel created for conversation openhermes-15026-0


 18%|█████████████████████▍                                                                                               | 15044/82022 [18:01:35<106:43:17,  5.74s/it]